In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from Load_from_postgres import load_data

In [2]:
# Load data from PostgreSQL
data = load_data()

# Check if data was loaded successfully
if data is not None:
    print("Data loaded successfully!")
    data.head() # Display the first few rows of the DataFrame
else:
    print("Failed to load data.")

Data loaded successfully!


In [3]:
data.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
data.isnull().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [5]:
def fill_missing_values(Data):
    for column in data.columns:
        if pd.api.types.is_numeric_dtype(data[column]):
            # Fill numeric columns with the mean
            mean_value = data[column].mean()
            data[column].fillna(mean_value, inplace=True)
        else:
            # Fill non-numeric columns with the mode
            mode_value = data[column].mode()[0]  # Get the first mode
            data[column].fillna(mode_value, inplace=True)

# Apply the function
fill_missing_values(data)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21268\724285368.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(mean_value, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21268\724285368.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

User Aggregation
aggregate average TCP retransmission, RTT, handset type, and throughput per customer.

In [6]:
# Aggregate TCP retransmission, RTT, and throughput per customer
experience_metrics = data.groupby('MSISDN/Number').agg({
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'TCP DL Retrans. Vol (Bytes)': 'sum',
    'TCP UL Retrans. Vol (Bytes)': 'sum',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean'
}).reset_index()

# View aggregated metrics
experience_metrics.head()


,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,3.360100e+10,46.000000,0.000000,2.080991e+07,759658.664811,37.0,39.0
1,3.360100e+10,30.000000,1.000000,2.080991e+07,759658.664811,48.0,51.0
2,3.360100e+10,109.795706,17.662883,2.080991e+07,759658.664811,48.0,49.0
3,3.360101e+10,69.000000,15.000000,1.066000e+03,759658.664811,204.0,44.0
4,3.360101e+10,57.000000,2.500000,3.015954e+07,780860.664811,20197.5,8224.5


Top and Bottom TCP, RTT, Throughput Metrics
listing the top 10 and bottom 10 customers based on TCP, RTT, and throughpu

In [8]:
# Top and bottom 10 customers by TCP, RTT, and throughput
top_10_tcp_dl = experience_metrics.nlargest(10, 'TCP DL Retrans. Vol (Bytes)')
bottom_10_tcp_dl = experience_metrics.nsmallest(10, 'TCP DL Retrans. Vol (Bytes)')

top_10_rtt_dl = experience_metrics.nlargest(10, 'Avg RTT DL (ms)')
bottom_10_rtt_dl = experience_metrics.nsmallest(10, 'Avg RTT DL (ms)')

top_10_throughput_dl = experience_metrics.nlargest(10, 'Avg Bearer TP DL (kbps)')
bottom_10_throughput_dl = experience_metrics.nsmallest(10, 'Avg Bearer TP DL (kbps)')

print("Top 10 TCP DL Retransmission:")
print(top_10_tcp_dl[['MSISDN/Number', 'TCP DL Retrans. Vol (Bytes)']])


Top 10 TCP DL Retransmission:
        MSISDN/Number  TCP DL Retrans. Vol (Bytes)
106853   4.188282e+10                 1.764264e+10
66860    3.366773e+10                 8.332733e+09
34123    3.365908e+10                 7.938179e+09
42366    3.366098e+10                 7.374101e+09
94681    3.376128e+10                 6.039192e+09
98213    3.376264e+10                 5.878002e+09
25774    3.365066e+10                 5.590344e+09
51638    3.366318e+10                 5.562845e+09
99929    3.376357e+10                 5.178607e+09
31965    3.365864e+10                 5.165785e+09


In [9]:
print("Bottom 10 TCP DL Retransmission:")
print(bottom_10_tcp_dl[['MSISDN/Number', 'TCP DL Retrans. Vol (Bytes)']])

Bottom 10 TCP DL Retransmission:
        MSISDN/Number  TCP DL Retrans. Vol (Bytes)
56622    3.366454e+10                          2.0
95322    3.376154e+10                          2.0
71705    3.366931e+10                          4.0
71778    3.366933e+10                          4.0
87156    3.369900e+10                          4.0
87348    3.369906e+10                          4.0
95213    3.376151e+10                          4.0
95331    3.376154e+10                          4.0
95774    3.376162e+10                          4.0
100616   3.376394e+10                          4.0


Throughput and TCP Analysis per Handset


analyzing throughput and TCP retransmission per handset type.

In [12]:
# Aggregate throughput and TCP retransmission by handset type
handset_analysis = data.groupby('Handset Type').agg({
    'Avg Bearer TP DL (kbps)': 'mean',
    'TCP DL Retrans. Vol (Bytes)': 'sum',
}).reset_index()

# View handset throughput and TCP retransmission analysis
handset_analysis.head()


,Handset Type,Avg Bearer TP DL (kbps),TCP DL Retrans. Vol (Bytes)
0,A-Link Telecom I. Cubot A5,23184.0,2.080991e+07
1,A-Link Telecom I. Cubot Note Plus,6527.0,6.023490e+05
2,A-Link Telecom I. Cubot Note S,8515.0,4.134448e+07
3,A-Link Telecom I. Cubot Nova,55667.0,1.358400e+05
4,A-Link Telecom I. Cubot Power,69126.0,7.799000e+03


Clustering (k=5) on Experience Metrics


Using k-means clustering on experience metrics to segment users

In [16]:
# Normalize experience metrics
scaler = StandardScaler()
experience_metrics_scaled = scaler.fit_transform(experience_metrics[['Avg RTT DL (ms)', 'TCP DL Retrans. Vol (Bytes)', 'Avg Bearer TP DL (kbps)']])

# Apply K-means clustering (k=5) for experience metrics
kmeans_exp = KMeans(n_clusters=5, random_state=42)
experience_metrics['Cluster'] = kmeans_exp.fit_predict(experience_metrics_scaled)

# View cluster assignments
experience_metrics.head()


,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Cluster
0,3.360100e+10,46.000000,0.000000,2.080991e+07,759658.664811,37.0,39.0,0
1,3.360100e+10,30.000000,1.000000,2.080991e+07,759658.664811,48.0,51.0,0
2,3.360100e+10,109.795706,17.662883,2.080991e+07,759658.664811,48.0,49.0,0
3,3.360101e+10,69.000000,15.000000,1.066000e+03,759658.664811,204.0,44.0,0
4,3.360101e+10,57.000000,2.500000,3.015954e+07,780860.664811,20197.5,8224.5,4
